In [16]:
import pandas as pd
import logging
from datetime import date, timedelta
from IPython.display import display


logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)


# File Paths

CALL_LOGS_PATH        = 'call_logs.csv'
AGENT_ROSTER_PATH     = 'agent_roster.csv'
DISPO_PATH            = 'disposition_summary.csv'
OUTPUT_PATH           = 'agent_performance_summary.csv'

# Helper Functions

def load_csv(path):
    df = pd.read_csv(path)
    logging.info(f"Loaded `{path}` ({len(df):,} rows)")
    return df

def validate_and_flag(df, name, required_cols):
    
    missing = set(required_cols) - set(df.columns)
    if missing:
        raise KeyError(f"{name}: missing columns {missing}")
    logging.info(f"{name}: required columns OK")
  
    for c in required_cols:
        n = df[c].isna().sum()
        if n:
            logging.warning(f"{name}: {n} missing values in `{c}`")
            display(df[df[c].isna()].head())
  
    if 'call_date' in df.columns:
        df['call_date'] = pd.to_datetime(df['call_date'], errors='raise').dt.date
    return df

def flag_duplicates(df, name, subset):
    n = df.duplicated(subset=subset).sum()
    if n:
        logging.warning(f"{name}: {n} duplicates on {subset}")
        display(df[df.duplicated(subset=subset, keep=False)].sort_values(subset).head())
    else:
        logging.info(f"{name}: no duplicates on {subset}")


# FETCH, VALIDATE & FLAG

call_logs           = load_csv(CALL_LOGS_PATH)
agent_roster        = load_csv(AGENT_ROSTER_PATH)
disposition_summary = load_csv(DISPO_PATH)

call_logs           = validate_and_flag(call_logs,           "call_logs",           ['call_id','agent_id','org_id','call_date'])
agent_roster        = validate_and_flag(agent_roster,        "agent_roster",        ['agent_id','org_id'])
disposition_summary = validate_and_flag(disposition_summary, "disposition_summary", ['agent_id','org_id','call_date'])

flag_duplicates(call_logs,           "call_logs",           ['call_id'])
flag_duplicates(agent_roster,        "agent_roster",        ['agent_id','org_id'])
flag_duplicates(disposition_summary, "disposition_summary", ['agent_id','org_id','call_date'])

# CLEANING

call_logs           = call_logs.dropna(subset=['call_id','agent_id','org_id','call_date'])
agent_roster        = agent_roster.dropna(subset=['agent_id','org_id'])
disposition_summary = disposition_summary.dropna(subset=['agent_id','org_id','call_date'])

call_logs['status'] = call_logs['status'].str.strip().str.lower()

call_logs           = call_logs.drop_duplicates(subset=['call_id'])
agent_roster        = agent_roster.drop_duplicates(subset=['agent_id','org_id'])
disposition_summary = disposition_summary.drop_duplicates(subset=['agent_id','org_id','call_date'])

call_logs = call_logs[call_logs['duration'] > 0]


# TRANSFORM (Feature Engineering)

calls_disp = pd.merge(
    call_logs, disposition_summary,
    on=['agent_id','org_id','call_date'],
    how='left',
    indicator=True
)
logging.info("call_logs ↔ dispo: %s", calls_disp['_merge'].value_counts().to_dict())
calls_disp = calls_disp.drop(columns=['_merge'])

full = pd.merge(
    calls_disp, agent_roster,
    on=['agent_id','org_id'],
    how='left',
    indicator=True
)
logging.info("… ↔ agent_roster: %s", full['_merge'].value_counts().to_dict())

full['completed'] = full['status'] == 'completed'
full['presence']  = full['login_time'].notna().astype(int)

agg = (
    full.groupby(['agent_id','org_id','call_date'], as_index=False)
        .agg(
            total_calls       = ('call_id',        'nunique'),
            unique_loans      = ('installment_id', 'nunique'),
            completed_calls   = ('completed',      'sum'),
            avg_duration_secs = ('duration',       'mean'),
            presence          = ('presence',       'max'),
        )
)
agg['connect_rate']     = (agg['completed_calls'] / agg['total_calls']).round(3)
agg['avg_duration_min'] = (agg['avg_duration_secs'] / 60).round(2)

agg = agg.merge(
    agent_roster[['agent_id','users_first_name','users_last_name','users_office_location']],
    on='agent_id', how='left'
)

# REPORT (Save + Slack Summary)

agg.to_csv(OUTPUT_PATH, index=False)
logging.info(f"Saved report to `{OUTPUT_PATH}`")

target_date = agg['call_date'].max()
today_df    = agg[agg['call_date'] == target_date]

if not today_df.empty:
    top = today_df.sort_values(['connect_rate','total_calls'], ascending=False).iloc[0]
    print(f"""
📊 *Agent Summary for {target_date}*
• *Top Performer:* {top.users_first_name} {top.users_last_name} ({top.connect_rate:.0%} connect rate)
• *Total Active Agents:* {today_df['agent_id'].nunique()}
• *Average Duration:* {today_df['avg_duration_min'].mean().round(2)} min
""")
else:
    print(f"No data for {target_date}")

# Preview results
display(agg.head())


2025-04-30 20:00:43,018 - INFO - Loaded `call_logs.csv` (500 rows)
2025-04-30 20:00:43,020 - INFO - Loaded `agent_roster.csv` (20 rows)
2025-04-30 20:00:43,021 - INFO - Loaded `disposition_summary.csv` (20 rows)
2025-04-30 20:00:43,022 - INFO - call_logs: required columns OK
2025-04-30 20:00:43,024 - INFO - agent_roster: required columns OK
2025-04-30 20:00:43,024 - INFO - disposition_summary: required columns OK
2025-04-30 20:00:43,026 - WARNING - call_logs: 11 duplicates on ['call_id']


,call_id,agent_id,org_id,installment_id,status,duration,created_ts,call_date
386,C1430,A016,O1,L1369,failed,4.30,2025-04-28T15:57:00,2025-04-28
275,C1430,A020,O2,L1623,failed,7.57,2025-04-28T00:59:00,2025-04-28
366,C2293,A014,O3,L1440,connected,14.42,2025-04-28T18:37:00,2025-04-28
212,C2293,A006,O1,L1624,no_answer,9.81,2025-04-28T21:09:00,2025-04-28
475,C2489,A016,O1,L1920,no_answer,14.90,2025-04-28T01:53:00,2025-04-28


2025-04-30 20:00:43,031 - INFO - agent_roster: no duplicates on ['agent_id', 'org_id']
2025-04-30 20:00:43,032 - INFO - disposition_summary: no duplicates on ['agent_id', 'org_id', 'call_date']
2025-04-30 20:00:43,049 - INFO - call_logs ↔ dispo: {'both': 489, 'left_only': 0, 'right_only': 0}
2025-04-30 20:00:43,064 - INFO - … ↔ agent_roster: {'both': 489, 'left_only': 0, 'right_only': 0}
2025-04-30 20:00:43,077 - INFO - Saved report to `agent_performance_summary.csv`



📊 *Agent Summary for 2025-04-28*
• *Top Performer:* AgentFirst3 AgentLast3 (38% connect rate)
• *Total Active Agents:* 20
• *Average Duration:* 0.12 min



,agent_id,org_id,call_date,total_calls,unique_loans,completed_calls,avg_duration_secs,presence,connect_rate,avg_duration_min,users_first_name,users_last_name,users_office_location
0,A001,O1,2025-04-28,20,20,2,6.843000,1,0.100,0.11,AgentFirst1,AgentLast1,Bangalore
1,A002,O1,2025-04-28,22,22,3,8.160909,1,0.136,0.14,AgentFirst2,AgentLast2,Delhi
2,A003,O1,2025-04-28,21,21,8,7.169524,1,0.381,0.12,AgentFirst3,AgentLast3,Mumbai
3,A004,O3,2025-04-28,27,27,4,8.056667,1,0.148,0.13,AgentFirst4,AgentLast4,Bangalore
4,A005,O3,2025-04-28,28,27,4,7.151786,0,0.143,0.12,AgentFirst5,AgentLast5,Bangalore
